In [1]:
# Necessary imports
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import glob
import re

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

print('Libraries imported')

Libraries imported


In [2]:
path = r'C:/Metis_Bootcamp/Regression_project/data/joined-data/' # data path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    data = pd.read_csv(filename, index_col=0, header=0)
    li.append(data)

df = pd.concat(li, axis=0, ignore_index=True)

print(df.shape)

df.head()

(21916, 15)


,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Entertainment,Safety,Price
0,2018 Toyota Highlander XLE,"36,996","Garden Grove, CA 92843",4.4,Gasoline,21,27,FWD,3.5L V6 24V PDI DOHC,Gray,Black,8-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']","$30,899"
1,2018 Jeep Grand Cherokee Limited,"28,499","Garden Grove, CA 92843",4.4,Gasoline,19,26,RWD,3.6L V6 24V MPFI DOHC,Bright White Clearcoat,Black,8-Speed Automatic,['Bluetooth'],"['Backup Camera', 'Brake Assist', 'Stability C...","$26,957"
2,2018 Honda Odyssey EX-L,"34,172","Anaheim, CA 92806",1.3,Gasoline,19,28,FWD,3.5L V6 24V GDI SOHC,White Diamond Pearl,Beige,9-Speed Automatic,"['Bluetooth', 'Apple CarPlay/Android Auto']","['Backup Camera', 'Brake Assist', 'Stability C...","$28,990"
3,2018 BMW X5 xDrive35i,"46,731","Anaheim, CA 92804",5.0,Gasoline,18,24,AWD,3.0L I6 24V GDI DOHC Turbo,Gray,White,8-Speed Automatic,['Bluetooth'],"['Brake Assist', 'Stability Control']","$37,995"
4,2020 Nissan Pathfinder SL,"27,206","Huntington Beach, CA 92647",3.7,Gasoline,20,27,FWD,3.5L V6 24V GDI DOHC,Brilliant Silver Metallic,Charcoal,Automatic CVT,['Bluetooth'],"['Brake Assist', 'Stability Control', 'Blind S...","$23,984"


In [3]:
def model_year_extractor(name):
    
    name_year = name.replace('Certified ','').strip()
    
    year = int(name_year.split(' ')[0])
    
    return year

In [4]:
df['Model Year'] = df['Name'].apply(model_year_extractor)

In [5]:
df['Model Year'].value_counts()

2018    6734
2019    3295
2020    3201
2017    2994
2016    1265
2015     911
2021     762
2014     657
2013     541
2012     401
2011     218
2010     156
2008     138
2007     105
2006      99
2005      82
2009      82
2004      64
2000      42
2003      40
2002      27
1999      21
2001      21
1998      14
1990      11
1997      11
1996       6
1991       3
1965       2
1988       2
1969       2
1983       2
1973       2
1972       1
1949       1
1985       1
1966       1
1957       1
Name: Model Year, dtype: int64

In [6]:
def make_extractor(name):
    
    if 'Land' in name:
        return 'Land Rover'
    
    elif 'Alfa' in name:
        return 'Alfa Romeo'
    
    elif 'MINI' in name:
        return 'MINI Cooper'
    
    elif 'Aston' in name:
        return 'Aston Martin'
    
    else:
        return name.split(' ')[2]

In [7]:
df['Make'] = df['Name'].apply(make_extractor)

In [8]:
df['Make'].value_counts()

Toyota           2458
Ford             1848
Mercedes-Benz    1712
BMW              1538
Honda            1435
Chevrolet        1289
Audi             1059
Jeep             1011
Lexus             940
Nissan            860
Hyundai           705
Subaru            650
Volkswagen        629
Mazda             584
GMC               503
Kia               480
Dodge             469
Acura             451
Land Rover        391
RAM               385
INFINITI          364
Porsche           299
Cadillac          254
Volvo             225
MINI Cooper       183
Chrysler          168
Tesla             148
Alfa Romeo        141
Buick             133
Jaguar            129
Mitsubishi        120
Lincoln           111
Maserati           82
Genesis            35
Scion              24
Bentley            18
FIAT               15
Rolls-Royce        13
Ferrari             9
McLaren             9
Pontiac             8
Aston Martin        5
Saturn              5
smart               5
Lamborghini         4
Isuzu     

In [9]:
def mileage_to_int(mileage):
    
    return int(mileage.replace(',',''))

def price_to_int(price):
    
    return int(price.replace('$','').replace(',',''))

In [10]:
#Converse mileage and price to int

df['Mileage'] = df['Mileage'].apply(mileage_to_int)

df['Price'] = df['Price'].apply(price_to_int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21916 entries, 0 to 21915
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            21916 non-null  object 
 1   Mileage         21916 non-null  int64  
 2   Address         21916 non-null  object 
 3   Rating          21898 non-null  float64
 4   Fuel Type       21916 non-null  object 
 5   City MPG        21916 non-null  int64  
 6   Highway MPG     21916 non-null  int64  
 7   Drivetrain      21916 non-null  object 
 8   Engine          21916 non-null  object 
 9   Exterior Color  21916 non-null  object 
 10  Interior Color  21916 non-null  object 
 11  Transmission    21916 non-null  object 
 12  Entertainment   21916 non-null  object 
 13  Safety          21916 non-null  object 
 14  Price           21916 non-null  int64  
 15  Model Year      21916 non-null  int64  
 16  Make            21916 non-null  object 
dtypes: float64(1), int64(5), object

In [11]:
df.Price.isna().sum()

0

In [12]:
df.describe()

,Mileage,Rating,City MPG,Highway MPG,Price,Model Year
count,21916.000000,21898.000000,21916.000000,21916.000000,2.191600e+04,21916.000000
mean,40059.076702,4.158814,22.099288,29.735308,3.017843e+04,2017.202820
std,34627.486168,0.892574,15.398344,24.295620,2.158660e+04,3.435621
min,1.000000,1.000000,-1.000000,-1.000000,9.950000e+02,1949.000000
25%,19729.000000,3.800000,17.000000,24.000000,1.959700e+04,2017.000000
50%,30380.000000,4.500000,21.000000,28.000000,2.690100e+04,2018.000000
75%,47732.000000,4.800000,24.000000,33.000000,3.645000e+04,2019.000000
max,385906.000000,5.000000,224.000000,384.000000,1.450000e+06,2021.000000


In [13]:
def num_ent_fea(entertainment):
    
    return len(entertainment.split(','))

def num_safe_fea(safety):
    
    return len(safety.split(','))

In [14]:
df['Entertainment'] = df['Entertainment'].apply(num_ent_fea)
df['Safety'] = df['Safety'].apply(num_safe_fea)

In [15]:
df.rename(columns={'Entertainment': 'Num_ent_features', 'Safety': 'Num_safe_features'}, inplace=True)

In [16]:
def define_transmission(transmission):
    
    if 'automatic' in transmission.lower():
        return 'Automatic'
    
    elif 'manual' in transmission.lower():
        return 'Manual'
    
    else:
        return 'Unknown'

In [17]:
df['Transmission'] = df['Transmission'].apply(define_transmission)

In [18]:
df['Transmission'].value_counts()

Automatic    21130
Manual         454
Unknown        332
Name: Transmission, dtype: int64

In [19]:
df['Transmission'].isna().sum()

0

In [20]:
df.head()

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Num_ent_features,Num_safe_features,Price,Model Year,Make
0,2018 Toyota Highlander XLE,36996,"Garden Grove, CA 92843",4.4,Gasoline,21,27,FWD,3.5L V6 24V PDI DOHC,Gray,Black,Automatic,1,2,30899,2018,Toyota
1,2018 Jeep Grand Cherokee Limited,28499,"Garden Grove, CA 92843",4.4,Gasoline,19,26,RWD,3.6L V6 24V MPFI DOHC,Bright White Clearcoat,Black,Automatic,1,3,26957,2018,Jeep
2,2018 Honda Odyssey EX-L,34172,"Anaheim, CA 92806",1.3,Gasoline,19,28,FWD,3.5L V6 24V GDI SOHC,White Diamond Pearl,Beige,Automatic,2,4,28990,2018,Honda
3,2018 BMW X5 xDrive35i,46731,"Anaheim, CA 92804",5.0,Gasoline,18,24,AWD,3.0L I6 24V GDI DOHC Turbo,Gray,White,Automatic,1,2,37995,2018,BMW
4,2020 Nissan Pathfinder SL,27206,"Huntington Beach, CA 92647",3.7,Gasoline,20,27,FWD,3.5L V6 24V GDI DOHC,Brilliant Silver Metallic,Charcoal,Automatic,1,3,23984,2020,Nissan


In [21]:
def define_engine(engine):
    
    if 'intercooled' in engine.lower():
        return 'Intercooled Turbo'
    
    elif 'dohc turbo' in engine.lower():
        return 'DOHC Turbo'
    
    elif 'twin turbo' in engine.lower():
        return 'Twin Turbo'
    
    elif 'hybrid' in engine.lower():
        return 'Hybrid'
    
    elif 'electric' in engine.lower():
        return 'Electric'
    
    else:
        return 'Regular'

In [22]:
df['Engine'] = df['Engine'].apply(define_engine)

In [23]:
df.Engine.value_counts()

Regular              12450
DOHC Turbo            5776
Twin Turbo            1305
Intercooled Turbo     1030
Hybrid                 891
Electric               464
Name: Engine, dtype: int64

In [24]:
def define_exterior_color(exterior_color):
    
    if 'black' in exterior_color.lower():
        return 'Black'
    
    elif 'white' in exterior_color.lower():
        return 'White'
    
    elif 'gray' in exterior_color.lower():
        return 'Gray'
    
    elif 'silver' in exterior_color.lower():
        return 'Silver'
    
    elif 'blue' in exterior_color.lower():
        return 'Blue'
    
    else:
        return 'Other'

In [25]:
df['Exterior Color'] = df['Exterior Color'].apply(define_exterior_color)

In [26]:
df['Exterior Color'].value_counts()

White     5164
Black     4998
Other     4523
Silver    2816
Gray      2634
Blue      1781
Name: Exterior Color, dtype: int64

In [27]:
def define_interior_color(interior_color):

    if 'ebony' in interior_color.lower() or 'ebony black' in interior_color.lower():
        return 'Ebony'
    
    elif 'black' in interior_color.lower():
        return 'Black'
    
    elif 'gray' in interior_color.lower():
        return 'Gray'
    
    elif 'charcoal' in interior_color.lower():
        return 'Charcoal'
    
    elif 'graphite' in interior_color.lower():
        return 'Graphite'
    
    else:
        return 'Other'

In [28]:
df['Interior Color'] = df['Interior Color'].apply(define_interior_color)

In [29]:
df['Interior Color'].value_counts()

Black       10799
Other        7172
Gray         2043
Ebony         998
Charcoal      498
Graphite      406
Name: Interior Color, dtype: int64

In [30]:
def define_drivetrain(drivetrain):
    
    if 'fwd' in drivetrain.lower() or 'front wheel drive' in drivetrain.lower() or 'front-wheel drive' in drivetrain.lower():
        return 'FWD'
    
    elif 'awd' in drivetrain.lower() or 'all wheel drive' in drivetrain.lower() or 'all-wheel drive' in drivetrain.lower():
        return 'AWD'
    
    elif '4wd' in drivetrain.lower() or 'four wheel drive' in drivetrain.lower() or 'four-wheel drive' in drivetrain.lower():
        return '4WD'
    
    elif 'rwd' in drivetrain.lower() or 'rear wheel drive' in drivetrain.lower() or 'rear-wheel drive' in drivetrain.lower():
        return 'RWD'
    
    else:
        return np.nan

In [31]:
df['Drivetrain'] = df['Drivetrain'].apply(define_drivetrain)

In [32]:
df['Drivetrain'].value_counts()

FWD    8079
AWD    6954
4WD    3662
RWD    3145
Name: Drivetrain, dtype: int64

In [33]:
      
df['City MPG'] = df['City MPG'].apply(lambda x: np.nan if x == -1 else x)

df['City MPG'].fillna(df['City MPG'].median(), inplace=True)

df['Highway MPG'] = df['Highway MPG'].apply(lambda x: np.nan if x == -1 else x)

df['Highway MPG'].fillna(df['Highway MPG'].median(), inplace=True)

In [34]:
df.dropna(inplace=True)

In [35]:
df.sample(10)

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Num_ent_features,Num_safe_features,Price,Model Year,Make
18322,2018 Lexus ES 350 Base,19580,"Tacoma, WA 98498",3.6,Gasoline,21.0,30.0,FWD,Regular,Black,Other,Automatic,1,5,28299,2018,Lexus
3264,2019 Dodge Charger SXT,30284,"Bellflower, CA 90706",4.5,Gasoline,19.0,30.0,RWD,Regular,Black,Black,Automatic,2,3,23260,2019,Dodge
4990,Certified 2018 Toyota Sienna LE,43423,"Claremont, CA 91711",4.5,Gasoline,19.0,27.0,FWD,Regular,White,Other,Automatic,1,5,26488,2018,Toyota
7641,2017 BMW 330 i,38273,"Newark, CA 94560",4.7,Gasoline,23.0,34.0,RWD,DOHC Turbo,Silver,Black,Automatic,1,3,21800,2017,BMW
7004,2018 Toyota Highlander LE,39265,"San Jose, CA 95136",4.7,Gasoline,19.0,26.0,AWD,Regular,Gray,Other,Automatic,1,5,37285,2018,Toyota
17833,2018 Tesla Model X 100D,11034,"Bellingham, WA 98225",2.3,Electric,86.0,89.0,AWD,Electric,Gray,Other,Automatic,2,2,77500,2018,Tesla
965,2019 Mazda Mazda3 AWD w/Select Package,15670,"Cerritos, CA 90703",4.2,Gasoline,25.0,33.0,AWD,Regular,Black,Black,Automatic,2,6,19888,2019,Mazda
16656,2017 Subaru Outback 2.5i Premium,50755,"Olympia, WA 98502",4.5,Gasoline,25.0,32.0,AWD,Regular,Other,Other,Automatic,1,6,21123,2017,Subaru
20061,2018 Honda CR-V EX-L,7794,"Lynnwood, WA 98037",4.7,Gasoline,27.0,33.0,AWD,DOHC Turbo,Silver,Gray,Automatic,1,4,29999,2018,Honda
21121,2018 Ford F-150 XLT,27464,"Seattle, WA 98125",4.7,Gasoline,20.0,22.0,RWD,Twin Turbo,Black,Gray,Automatic,1,3,33995,2018,Ford


In [36]:
#df.to_csv('data/cleaned_10000.csv')

In [37]:
df.describe()

,Mileage,Rating,City MPG,Highway MPG,Num_ent_features,Num_safe_features,Price,Model Year
count,21823.000000,21823.000000,21823.000000,21823.000000,21823.000000,21823.000000,2.182300e+04,21823.000000
mean,39984.109517,4.157801,23.651331,31.857536,1.720020,3.539981,3.016448e+04,2017.222105
std,34508.127717,0.893225,14.066905,22.832033,0.702436,1.429344,2.155784e+04,3.335696
min,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,9.950000e+02,1949.000000
25%,19737.000000,3.800000,18.000000,25.000000,1.000000,3.000000,1.959800e+04,2017.000000
50%,30353.000000,4.500000,21.000000,29.000000,2.000000,3.000000,2.690100e+04,2018.000000
75%,47667.500000,4.800000,24.000000,33.000000,2.000000,5.000000,3.641750e+04,2019.000000
max,385906.000000,5.000000,224.000000,384.000000,4.000000,7.000000,1.450000e+06,2021.000000


In [38]:
df.shape

(21823, 17)

In [39]:
df.head()

,Name,Mileage,Address,Rating,Fuel Type,City MPG,Highway MPG,Drivetrain,Engine,Exterior Color,Interior Color,Transmission,Num_ent_features,Num_safe_features,Price,Model Year,Make
0,2018 Toyota Highlander XLE,36996,"Garden Grove, CA 92843",4.4,Gasoline,21.0,27.0,FWD,Regular,Gray,Black,Automatic,1,2,30899,2018,Toyota
1,2018 Jeep Grand Cherokee Limited,28499,"Garden Grove, CA 92843",4.4,Gasoline,19.0,26.0,RWD,Regular,White,Black,Automatic,1,3,26957,2018,Jeep
2,2018 Honda Odyssey EX-L,34172,"Anaheim, CA 92806",1.3,Gasoline,19.0,28.0,FWD,Regular,White,Other,Automatic,2,4,28990,2018,Honda
3,2018 BMW X5 xDrive35i,46731,"Anaheim, CA 92804",5.0,Gasoline,18.0,24.0,AWD,DOHC Turbo,Gray,Other,Automatic,1,2,37995,2018,BMW
4,2020 Nissan Pathfinder SL,27206,"Huntington Beach, CA 92647",3.7,Gasoline,20.0,27.0,FWD,Regular,Silver,Charcoal,Automatic,1,3,23984,2020,Nissan


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21823 entries, 0 to 21915
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               21823 non-null  object 
 1   Mileage            21823 non-null  int64  
 2   Address            21823 non-null  object 
 3   Rating             21823 non-null  float64
 4   Fuel Type          21823 non-null  object 
 5   City MPG           21823 non-null  float64
 6   Highway MPG        21823 non-null  float64
 7   Drivetrain         21823 non-null  object 
 8   Engine             21823 non-null  object 
 9   Exterior Color     21823 non-null  object 
 10  Interior Color     21823 non-null  object 
 11  Transmission       21823 non-null  object 
 12  Num_ent_features   21823 non-null  int64  
 13  Num_safe_features  21823 non-null  int64  
 14  Price              21823 non-null  int64  
 15  Model Year         21823 non-null  int64  
 16  Make               218

In [41]:
df.to_csv('data/clean_20000.csv')